<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/Lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дополнительные ссылки:
1. https://sales-generator.ru/blog/rekomendatelnye-sistemy/
2. https://habr.com/ru/companies/prequel/articles/567648/
3. https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101
4. https://habr.com/ru/companies/prequel/articles/573880/

https://bit.ly/RSML-3-colab

## Коллаборативная фильтрация

### item-to-item

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-06-05 15:16:46--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.23.100, 74.125.23.102, 74.125.23.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.23.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lsc6ba6amsn2r1sitn9lnv9op3p1hfqt/1685978175000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=4b7514fe-bc05-45ef-afec-e00aef41eb8d [following]
--2023-06-05 15:16:48--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lsc6ba6amsn2r1sitn9lnv9op3p1hfqt/1685978175000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=4b7514fe-bc05-45ef-afec-e00aef41eb8d
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 64.233.187.132, 2404:6800:4008:c04::84
Connecting to doc-0g-84-

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movies_with_ratings.title.value_counts()

Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: title, Length: 9719, dtype: int64

In [ ]:
movies_with_ratings.userId.value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: userId, Length: 610, dtype: int64

In [ ]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

610

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)

    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

100%|██████████| 9719/9719 [00:02<00:00, 3800.48it/s]


In [ ]:
movie_vector['Toy Story (1995)']

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

In [ ]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

100%|██████████| 9719/9719 [00:02<00:00, 4553.26it/s]


9718

In [ ]:
distances[:10]

[0.9223029947150825,
 0.9300726952435743,
 0.950553928616633,
 1.0,
 1.0,
 0.968921197886033,
 0.877349727371666,
 0.9378423957720661,
 0.7578680102442159,
 0.9065861273541315]

In [ ]:
best_indexes = np.argsort(distances)[:10]
best_indexes

array([5511, 5571,  420, 4736, 5203,  405, 6864, 5204, 4322, 4737])

In [ ]:
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

('Matrix, The (1999)', 0.28606257328028306)
('Memento (2000)', 0.33040704494686757)
('American History X (1998)', 0.3509458374819995)
('Kill Bill: Vol. 1 (2003)', 0.360262413456623)
('Lord of the Rings: The Fellowship of the Ring, The (2001)', 0.36425580638414523)
('American Beauty (1999)', 0.3744505150386159)
('Pulp Fiction (1994)', 0.37678014788325576)
('Lord of the Rings: The Return of the King, The (2003)', 0.37798413607448667)
('Inception (2010)', 0.3845825728533704)
('Kill Bill: Vol. 2 (2004)', 0.38565615909903594)


### Surprise

https://surpriselib.com/

- Алгоритмы https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095482 sha256=1fb9739ea54c91e04b26383cb9edf5e5c63cf96aafa635936693d90cdc77126c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
ratings.rating.min()

0.5

In [ ]:
ratings.rating.max()

5.0

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

#### user_based

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [ ]:
test_pred[:3]

[Prediction(uid=542, iid='Kill Bill: Vol. 1 (2003)', r_ui=2.5, est=3.7592260572900007, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=469, iid='Bridge on the River Kwai, The (1957)', r_ui=4.0, est=4.168241951992395, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=317, iid='Scarface (1983)', r_ui=5.0, est=4.062599058241541, details={'actual_k': 49, 'was_impossible': False})]

In [ ]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.474085079084162, details={'actual_k': 50, 'was_impossible': False})

#### item_based

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)

In [ ]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8982


0.898181158065249

In [ ]:
new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [ ]:
accuracy.rmse(new_pred, verbose=True)

TypeError: ignored

In [ ]:
new_pred = algo.predict(uid=20000, iid='Fight Club (1999)')
new_pred

Prediction(uid=20000, iid='Fight Club (1999)', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
new_pred = algo.predict(uid=2, iid='unkown')
new_pred

Prediction(uid=2, iid='unkown', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
trainset.global_mean

3.498958694897605

#### Рекомендации

In [ ]:
def generate_recommendation(uid, model, dataset, thresh=4.5, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)

    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))

            if len(rec_list) >= amount:
                return rec_list

In [ ]:
generate_recommendation(2, algo, dataset, thresh=4.8)

[('Duel in the Sun (1946)', 5.0),
 ('Hype! (1996)', 5.0),
 ('Man with the Golden Arm, The (1955)', 5.0),
 ('Four Seasons, The (1981)', 5.0),
 ('Front of the Class (2008)', 5.0)]